In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

2024-07-22 13:20:12.258634: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 13:20:12.258696: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 13:20:12.260179: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class FER2013Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
        self.images, self.labels = self._load_dataset()

    def _load_dataset(self):
        images = []
        labels = []
        for class_idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(self.root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                images.append(img_path)
                labels.append(class_idx)
        return images, labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # CLIP expects 224x224
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = FER2013Dataset('/kaggle/input/fer2013/train',
                               transform=transform)
test_dataset = FER2013Dataset('/kaggle/input/fer2013/test',
                              transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,)

In [4]:
class FERModel(nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super(FERModel, self).__init__()
        self.clip_model = (CLIPModel.from_pretrained("openai/clip-vit-base-patch32").vision_model).to(device)
        self.classifier = (nn.Linear(768, num_classes)).to(device) # CLIP's output dimension is 768
        # Freeze CLIP parameters
        for param in self.clip_model.parameters():
            param.requires_grad = False

    def forward(self, x):
        with torch.no_grad():
            features = self.clip_model(x).last_hidden_state[:, 0, :]  # Use CLS token
        output = self.classifier(features)
        return output


In [5]:
def train(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")
    for batch_idx, (images, labels) in progress_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        progress_bar.set_postfix({
            'Loss': f"{running_loss/(batch_idx+1):.3f}",
            'Acc': f"{100.*correct/total:.2f}%"
        })    
    return running_loss / len(train_loader), 100. * correct / total

In [6]:
def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    accuracy = 100. * correct / total
    average_loss = total_loss / len(test_loader)
    return average_loss, accuracy

In [7]:
model = FERModel(num_classes=7)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
best_accuracy = 0.0

num_epochs = 20
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device, epoch)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    
    # Store metrics
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print()


Evaluating: 100%|██████████| 225/225 [00:35<00:00,  6.26it/s]


Epoch 1/20:
Train Loss: 1.1243, Train Acc: 59.40%
Test Loss: 1.0103, Test Acc: 61.86%



Evaluating: 100%|██████████| 225/225 [00:35<00:00,  6.35it/s]


Epoch 2/20:
Train Loss: 0.9550, Train Acc: 64.83%
Test Loss: 0.9688, Test Acc: 62.76%



Evaluating: 100%|██████████| 225/225 [00:35<00:00,  6.39it/s]


...

Epoch 18/20:
Train Loss: 0.8333, Train Acc: 68.93%
Test Loss: 0.9021, Test Acc: 65.90%



Evaluating: 100%|██████████| 225/225 [00:34<00:00,  6.51it/s]


Epoch 19/20:
Train Loss: 0.8307, Train Acc: 68.98%
Test Loss: 0.9061, Test Acc: 65.55%



Evaluating: 100%|██████████| 225/225 [00:34<00:00,  6.50it/s]

Epoch 20/20:
Train Loss: 0.8291, Train Acc: 69.37%
Test Loss: 0.9012, Test Acc: 65.83%



In [10]:
torch.save(model, '/kaggle/working/model.pth')

In [ ]:
model = torch.load('model.pth')
